In [ ]:
import pandas as pd
import random

In [ ]:
station_info = pd.read_csv('../output/zcm/station_infos100000.csv')
is_weekday = True
station_info['time'] = pd.to_datetime(station_info['time'])
if is_weekday:
    station_info = station_info[station_info['time'].dt.weekday > 4]
else:
    station_info = station_info[station_info['time'].dt.weekday < 5]

In [ ]:
def mape_hour(station_info):
    station_info = station_info[station_info['num_current_car']>0]
    # 仿真步长
    step_length = 5*60

    import ast
    station_info_table = station_info[['station_id','lon','lat','max_capacity','charge_speed_station']].drop_duplicates().copy()
    station_info = station_info[['station_id','time','current_car','waiting_car']]

    station_info['time'] = pd.to_datetime(station_info['time'])
    station_info['current_car'] = station_info['current_car'].apply(lambda a:ast.literal_eval(a))
    station_info['waiting_car'] = station_info['waiting_car'].apply(lambda a:ast.literal_eval(a))
    station_info.sort_values(by=['station_id','time'],inplace=True)


    # 将current_car和waiting_car列转换为车辆记录的列表
    current_car_infos = station_info[['station_id','time','current_car']].explode('current_car')
    current_car_infos = current_car_infos[~current_car_infos['current_car'].isnull()]
    current_car_infos = current_car_infos.sort_values(by=['current_car','time'])[['current_car','time','station_id']]
    waiting_car_infos = station_info[['station_id','time','waiting_car']].explode('waiting_car')
    waiting_car_infos = waiting_car_infos[~waiting_car_infos['waiting_car'].isnull()]
    waiting_car_infos = waiting_car_infos.sort_values(by=['waiting_car','time'])[['waiting_car','time','station_id']]
    def get_charging_order(current_car_infos):
        current_car_infos['timegap'] = current_car_infos['time'].diff().dt.total_seconds().fillna(1000000).astype(int)
        current_car_infos['order_id'] = (current_car_infos['timegap']>step_length).cumsum()
        charge_info_s = current_car_infos.groupby(['current_car','order_id']).first().reset_index()
        charge_info_e = current_car_infos.groupby(['current_car','order_id']).last().reset_index()
        charging_order = pd.merge(charge_info_s,charge_info_e,on=['current_car','order_id','station_id'])
        charging_order
        charging_order = charging_order[['current_car','order_id','time_x','time_y','station_id']]
        charging_order.columns = ['carid','order_id','stime','etime','station_id']
        charging_order['duration'] = (charging_order['etime']-charging_order['stime']).dt.total_seconds()
        charging_order = charging_order[charging_order['duration']>0]
        return charging_order
    
    # 计算充电信息
    charging_orders = get_charging_order(current_car_infos)
    # 计算等待信息
    waiting_orders =  get_charging_order(waiting_car_infos.rename(columns={'waiting_car':'current_car'}))
    # 保存充电信息和等待信息
    charging_orders = pd.merge(charging_orders,station_info_table)
    waiting_orders = pd.merge(waiting_orders,station_info_table)
    # 计算充电时长
    charging_orders['duration'] = (charging_orders['etime']-charging_orders['stime']).dt.total_seconds()
    waiting_orders['duration'] = (waiting_orders['etime']-waiting_orders['stime']).dt.total_seconds()
    # grid=pd.read_csv('./mape/grid.csv')
    charging_orders['day'] = charging_orders['stime'].dt.day
    charging_orders['hour'] = charging_orders['stime'].dt.hour
    charging_orders_hour_agg = charging_orders.groupby(['station_id','hour','lon','lat'])['carid'].count().reset_index()
    charging_orders_hour_agg.rename(columns={'carid':'count'},inplace=True)
    charging_orders_hour_agg.sort_values(by=['station_id','hour'],inplace=True)
    return charging_orders_hour_agg
charge_order = mape_hour(station_info)
charge_order.rename(columns={'station_id':'stationId'},inplace=True) 

In [ ]:
import geopandas as gpd
sta_grid = gpd.read_file(f'../pred_data/zcm_grid_100_{type}.geojson')
type = 'holiday'
if not is_weekday:
    type = 'workday'
truth_order = pd.read_csv(f'../pred_data/untrain_100_{type}.csv')
untrain_grid = truth_order[['LONCOL','LATCOL']]
untrain_grid.drop_duplicates(subset=['LONCOL','LATCOL'],keep='first',inplace=True)
untrain_grid['type'] = 1
untrain_sta = pd.merge(sta_grid,untrain_grid,on=['LONCOL','LATCOL'],how='left')
untrain_sta['type'] = untrain_sta['type'].fillna(0)
untrain_sta = untrain_sta[untrain_sta['type']==1]
untrain_sta

In [ ]:
charge_order_untrain = pd.merge(charge_order,untrain_sta[['LONCOL','LATCOL','stationId','type']],how='left')
charge_order_untrain = charge_order_untrain[charge_order_untrain['type']==1]
charge_order_untrain = charge_order_untrain.groupby(['LONCOL','LATCOL','hour'])['count'].sum().reset_index()
charge_order_untrain

In [ ]:
output_merge = pd.merge(truth_order,charge_order_untrain,on=['LONCOL','LATCOL','hour'],how='left')
output_merge['count'] = output_merge['count'].fillna(0)
output_merge

In [ ]:
output_merge['order'].min(), output_merge['order'].max(), output_merge['count'].min(), output_merge['count'].max()

In [ ]:
output_merge['count_normalized'] = (output_merge['count'] - output_merge['count'].min()) / (output_merge['count'].max() - output_merge['count'].min())
# 归一化 'order' 列
output_merge['order_normalized'] = (output_merge['order'] - output_merge['order'].min()) / (output_merge['order'].max() - output_merge['order'].min())

In [ ]:
output_merge

In [ ]:
output_merge['abm_dif'] = abs(output_merge['count_normalized']-output_merge['order_normalized'])
output_merge['dif'] = output_merge.apply(lambda x: 0 if x['order_normalized']==0 else (x['abm_dif']/x['order_normalized']),axis=1)


In [ ]:
output_merge['dif'] = output_merge['dif']*100
output_merge

In [86]:
output_merge

,hour,LONCOL,LATCOL,order,pred_order,dif_per,count,count_normalized,order_normalized,abm_dif,dif
0,1,109,104,1,1.17,17.000000,1.0,0.083333,0.000000,0.083333,0.000000
1,8,109,104,1,1.57,57.000000,3.0,0.250000,0.000000,0.250000,0.000000
2,9,109,104,2,1.66,17.000000,1.0,0.083333,0.018182,0.065152,358.333333
3,10,109,104,2,1.76,12.000000,1.0,0.083333,0.018182,0.065152,358.333333
4,14,109,104,3,3.84,28.000000,1.0,0.083333,0.036364,0.046970,129.166667
...,...,...,...,...,...,...,...,...,...,...,...
980,6,176,120,1,3.19,219.000000,0.0,0.000000,0.000000,0.000000,0.000000
981,9,176,120,4,6.10,52.500000,0.0,0.000000,0.054545,0.054545,100.000000
982,17,176,120,13,11.00,15.384615,0.0,0.000000,0.218182,0.218182,100.000000
983,18,176,120,12,10.03,16.416667,0.0,0.000000,0.200000,0.200000,100.000000


In [ ]:
output_merge['abm_dif'].mean()